# 🎓 Lesson 2.1: Understanding Tools

## 📚 What You'll Learn

By the end of this lesson, you'll understand:
- What tools are and why they're powerful
- How to define a tool for Claude
- The tool use loop (request → execute → respond)
- Building your first calculator tool
- Handling tool responses

**Time to Complete**: 45-60 minutes

---

## 🛠️ What Are Tools?

Imagine Claude as a brilliant assistant who can THINK but can't DO:

**Without Tools**:
- User: "What's 123,456 × 789?"
- Claude: "Let me think... approximately 97,406,784" ❌ (might be wrong!)

**With Tools**:
- User: "What's 123,456 × 789?"
- Claude: "I need to use my calculator tool!" 🛠️
- *Uses calculator: 123456 * 789*
- Calculator: 97,406,784
- Claude: "The answer is exactly 97,406,784" ✅ (100% accurate!)

### Tools Let Claude:
- 🧮 **Calculate** precisely (no estimation!)
- 🌐 **Search** the web for current info
- 📧 **Send** emails
- 💾 **Save** files
- 🗄️ **Query** databases  
- 📞 **Call** APIs
- 🎨 **Do anything** you can code!

---

## 🔄 The Tool Use Loop

Here's how it works:

```
1. USER: Asks a question
         ↓
2. CLAUDE: "I need to use a tool!"
          Returns: tool_use request
         ↓
3. YOUR CODE: Executes the tool
             Returns: result
         ↓
4. CLAUDE: Uses the result
          Returns: final answer
         ↓
5. USER: Gets the answer!
```

---

## 🚀 Setup

In [ ]:
import os
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()
client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

print("✅ Client ready for tool use!")

## 🔧 Defining Your First Tool

A tool definition has 3 parts:
1. **Name**: What the tool is called
2. **Description**: What it does (Claude reads this!)
3. **Input Schema**: What parameters it needs

Let's define a calculator tool!

In [ ]:
# Define a calculator tool
calculator_tool = {
    "name": "calculator",
    "description": "Performs basic arithmetic operations. Use this for any math calculations. Supports addition, subtraction, multiplication, and division.",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The mathematical operation to perform"
            },
            "num1": {
                "type": "number",
                "description": "The first number"
            },
            "num2": {
                "type": "number",
                "description": "The second number"
            }
        },
        "required": ["operation", "num1", "num2"]
    }
}

# LINE-BY-LINE EXPLANATION:
# ---------------------------
#
# "name": "calculator"
#   This is the unique identifier for your tool
#   Claude will use this name when requesting the tool
#   Use lowercase, no spaces (use underscores instead)
#
# "description": "Performs basic arithmetic..."
#   This is CRUCIAL - Claude reads this to decide WHEN to use the tool
#   Be specific about what it does and when to use it
#   Think of it as instructions for Claude
#
# "input_schema": {...}
#   This defines what parameters the tool accepts
#   Uses JSON Schema format
#   "type": "object" means it takes an object with properties
#
# "properties": {...}
#   Each property is a parameter the tool accepts
#   Each has a type (string, number, boolean, etc.)
#   And a description (helps Claude use it correctly)
#
# "enum": ["add", "subtract", ...]
#   Limits the values to only these options
#   Claude can ONLY choose from this list
#   Great for ensuring valid inputs
#
# "required": ["operation", "num1", "num2"]
#   These parameters MUST be provided
#   Claude will always include them when using the tool

print("✅ Calculator tool defined!")
print("\nTool definition:")
import json
print(json.dumps(calculator_tool, indent=2))

## 💬 Making a Tool-Enabled API Call

Now let's ask Claude a math question and see it request the tool!

In [ ]:
# Ask Claude a math question
response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    tools=[calculator_tool],  # Give Claude access to our tool!
    messages=[
        {"role": "user", "content": "What is 42 multiplied by 17?"}
    ]
)

# LINE-BY-LINE EXPLANATION:
# ---------------------------
# tools=[calculator_tool]
#   This is a NEW parameter!
#   It's a LIST of tool definitions
#   Claude can now "see" and use these tools
#   You can provide multiple tools at once

print("Claude's response:")
print(response)
print("\n" + "="*60 + "\n")

# Look at the content
print("Response content:")
for content_block in response.content:
    print(f"Type: {content_block.type}")
    if content_block.type == "tool_use":
        print(f"Tool name: {content_block.name}")
        print(f"Tool input: {content_block.input}")
    else:
        print(f"Text: {content_block.text}")

### 🔍 Understanding the Response

Notice that Claude returned a `tool_use` block instead of text!

The response contains:
- **type**: `"tool_use"` (not `"text"`!)
- **name**: `"calculator"` (which tool to use)
- **input**: `{"operation": "multiply", "num1": 42, "num2": 17}` (the parameters)
- **id**: A unique identifier for this tool use

Now WE need to actually run the calculator and give Claude the result!

---

## ⚙️ Implementing the Tool Function

Let's write the actual calculator function that does the math.

In [ ]:
def run_calculator(operation, num1, num2):
    """
    Execute a calculator operation.
    
    Args:
        operation (str): The operation to perform
        num1 (float): First number
        num2 (float): Second number
    
    Returns:
        dict: Result of the calculation
    """
    if operation == "add":
        result = num1 + num2
    elif operation == "subtract":
        result = num1 - num2
    elif operation == "multiply":
        result = num1 * num2
    elif operation == "divide":
        if num2 == 0:
            return {"error": "Cannot divide by zero"}
        result = num1 / num2
    else:
        return {"error": f"Unknown operation: {operation}"}
    
    return {"result": result}

# LINE-BY-LINE EXPLANATION:
# ---------------------------
# This is just a regular Python function
# It takes the parameters that Claude provided
# Does the actual work (math calculation)
# Returns the result as a dictionary
#
# The key insight: Claude DOESN'T do the math
# Claude just ASKS us to do it via tool_use
# We run the function and send back the result
#
# You could do ANYTHING here:
# - Call an API
# - Query a database
# - Send an email
# - Read a file
# - Process an image
# - Literally anything Python can do!

# Test it!
print("Testing calculator function:")
print(run_calculator("multiply", 42, 17))
print(run_calculator("add", 10, 5))
print(run_calculator("divide", 10, 0))  # Error handling!

## 🔄 Completing the Tool Loop

Now let's complete the full cycle:
1. User asks question
2. Claude requests tool
3. We execute tool
4. We send result back to Claude
5. Claude gives final answer

In [ ]:
# STEP 1: User asks a question
user_message = "What is 42 multiplied by 17?"

# STEP 2: Claude requests tool
response1 = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    tools=[calculator_tool],
    messages=[{"role": "user", "content": user_message}]
)

print("STEP 2: Claude's response")
print(f"Stop reason: {response1.stop_reason}")  # Will be 'tool_use'

# Extract tool use from response
tool_use = None
for block in response1.content:
    if block.type == "tool_use":
        tool_use = block
        break

if tool_use:
    print(f"\nClaude wants to use: {tool_use.name}")
    print(f"With input: {tool_use.input}")
    print("\n" + "="*60 + "\n")
    
    # STEP 3: Execute the tool
    tool_result = run_calculator(
        operation=tool_use.input["operation"],
        num1=tool_use.input["num1"],
        num2=tool_use.input["num2"]
    )
    
    print("STEP 3: Tool result")
    print(tool_result)
    print("\n" + "="*60 + "\n")
    
    # STEP 4: Send result back to Claude
    response2 = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        tools=[calculator_tool],
        messages=[
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response1.content},  # Claude's tool request
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result)
                    }
                ]
            }
        ]
    )
    
    # LINE-BY-LINE EXPLANATION:
    # ---------------------------
    # messages=[
    #     {"role": "user", "content": user_message},
    #       ↑ Original question
    #
    #     {"role": "assistant", "content": response1.content},
    #       ↑ Claude's tool request (we include it in conversation history)
    #
    #     {"role": "user", "content": [{"type": "tool_result", ...}]}
    #       ↑ Our tool result (sent as a user message)
    # ]
    #
    # "type": "tool_result"
    #   This tells Claude "here's the result of the tool you requested"
    #
    # "tool_use_id": tool_use.id
    #   This matches the result to the specific tool request
    #   Important if Claude uses multiple tools!
    #
    # "content": str(tool_result)
    #   The actual result (converted to string)
    #   Can be JSON, plain text, whatever makes sense
    
    # STEP 5: Claude's final answer
    print("STEP 5: Claude's final answer")
    print(response2.content[0].text)

### 🎉 You Did It!

You just completed a full tool use loop!

1. ✅ Asked Claude a question
2. ✅ Claude requested a tool
3. ✅ You executed the tool
4. ✅ You sent back the result
5. ✅ Claude gave you the final answer

---

## 🎯 Building a Reusable Tool Handler

Let's make this easier with a helper function!

In [ ]:
def use_tools(user_message, tools, tool_functions):
    """
    Handle the complete tool use loop automatically.
    
    Args:
        user_message (str): The user's question
        tools (list): List of tool definitions
        tool_functions (dict): Mapping of tool names to Python functions
    
    Returns:
        str: Claude's final answer
    """
    messages = [{"role": "user", "content": user_message}]
    
    # Keep looping until Claude gives a final answer (no more tools)
    while True:
        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1024,
            tools=tools,
            messages=messages
        )
        
        # Check if Claude used a tool
        if response.stop_reason == "tool_use":
            # Add Claude's response to messages
            messages.append({"role": "assistant", "content": response.content})
            
            # Execute all requested tools
            tool_results = []
            for block in response.content:
                if block.type == "tool_use":
                    print(f"🛠️ Using tool: {block.name}")
                    print(f"   Input: {block.input}")
                    
                    # Call the appropriate function
                    tool_func = tool_functions[block.name]
                    result = tool_func(**block.input)  # Unpack input as kwargs
                    
                    print(f"   Result: {result}")
                    
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": block.id,
                        "content": str(result)
                    })
            
            # Add tool results to messages
            messages.append({"role": "user", "content": tool_results})
            
        else:
            # Claude gave a final answer (no more tools needed)
            return response.content[0].text

print("✅ use_tools() helper function ready!")

In [ ]:
# Now using tools is EASY!
answer = use_tools(
    user_message="What is 1234 divided by 56?",
    tools=[calculator_tool],
    tool_functions={"calculator": run_calculator}
)

print("\n" + "="*60)
print("Final Answer:")
print(answer)
print("="*60)

## 🎯 Practice Exercise 1: Weather Tool

**Task**: Create a (simulated) weather tool.

In [ ]:
# YOUR TURN: Define a weather tool

weather_tool = {
    "name": "get_weather",
    "description": "Get the current weather for a city. Returns temperature, conditions, and humidity.",
    "input_schema": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city name"
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Temperature unit"
            }
        },
        "required": ["city"]
    }
}

# Implement the weather function (simulated)
def get_weather(city, unit="celsius"):
    """Simulate getting weather (in real app, this would call an API)."""
    # Fake weather data
    import random
    temp = random.randint(15, 30) if unit == "celsius" else random.randint(60, 85)
    conditions = random.choice(["sunny", "cloudy", "rainy", "partly cloudy"])
    
    return {
        "city": city,
        "temperature": temp,
        "unit": unit,
        "conditions": conditions,
        "humidity": random.randint(30, 80)
    }

# Test it!
answer = use_tools(
    user_message="What's the weather like in Paris?",
    tools=[weather_tool],
    tool_functions={"get_weather": get_weather}
)

print("\nFinal Answer:")
print(answer)

## 🎯 Practice Exercise 2: Multiple Tools

**Task**: Give Claude access to BOTH calculator and weather!

In [ ]:
# Use multiple tools at once
answer = use_tools(
    user_message="What's the weather in Tokyo? Also, what's 15 times 23?",
    tools=[calculator_tool, weather_tool],  # Multiple tools!
    tool_functions={
        "calculator": run_calculator,
        "get_weather": get_weather
    }
)

print("\nFinal Answer:")
print(answer)

## ✅ Lesson Complete!

### What You Learned:
- ✅ Tools let Claude DO things (not just think)
- ✅ Tool definitions have: name, description, input_schema
- ✅ The tool loop: request → execute → respond
- ✅ `stop_reason == "tool_use"` means Claude wants a tool
- ✅ You execute tools, Claude uses results
- ✅ Can give Claude multiple tools at once

### Next Steps:
📖 **Lesson 2.2**: Building Your First Agent - Create an agent that uses tools autonomously!

---

## 🤔 Reflection Questions

1. Why is the tool description important?
2. What's the difference between `tool_use` and `tool_result`?
3. Who actually executes tools - Claude or your code?
4. Can Claude use multiple tools in one request?
5. What happens if you don't send back a tool_result?

Ready for more? Open `lesson_2.2.ipynb`!